In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 22902726
paper_name = 'schmidt_boyer_2012' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/BA sensitivity comprehensive data file.xlsx', sheet_name='Sheet1', skiprows=1)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 620 x 7


In [10]:
original_data.head()

,"Mutants sensitive in liquid screen (Mat alpa, 0.4%, contains duplicates)","Mutants sensitive in liquid screen (Mat alpha, 0.4%, duplicates removed)","Mutants sensitive in solid screen (Mat alpha, contains duplicates)",Unnamed: 3,Mutants also sensitive in Mat A (duplicates removed),Standard name,MIC
0,YAL044C,YAL044C,YAL044C,NaN,YAL044C,GCV3,0.3
1,YAL026C,YAL026C,YAL026C,NaN,YAL026C,DRS2,0.2
2,YAL013W,YAL013W,YAL013W,NaN,YAL013W,DEP1,0.3
3,YAL010C,YAL010C,YAL010C,NaN,YAL002W,VPS8,0.3
4,YAL009W,YAL009W,YAL009W,NaN,YLR113W,HOG1,0.2


In [11]:
original_data['orf'] = original_data['Mutants sensitive in liquid screen (Mat alpa, 0.4%, contains duplicates)'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Mutants sensitive in liquid screen (Mat alpa, 0.4%, contains duplicates), Mutants sensitive in liquid screen (Mat alpha, 0.4%, duplicates removed), Mutants sensitive in solid screen (Mat alpha, contains duplicates), Unnamed: 3, Mutants also sensitive in Mat A (duplicates removed), Standard name, MIC, orf]
Index: []


In [15]:
original_data['data'] = -1

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(611, 1)

# Load & process tested strains

In [20]:
tested = pd.read_excel('raw_data/BA sensitivity comprehensive data file.xlsx', sheet_name='Strain list')

In [21]:
tested.head()

,record no.,ORF name,strain,batch,plate,row,column,OD BA/OD kan
0,10338.0,YAL068C,BY4739,01_1,101.0,A,2,NaN
1,10339.0,YAL067C,BY4739,01_1,101.0,A,3,NaN
2,10340.0,YAL066W,BY4739,01_1,101.0,A,4,NaN
3,10341.0,YAL065C,BY4739,01_1,101.0,A,5,NaN
4,10345.0,YAL062W,BY4739,01_1,101.0,A,6,NaN


In [22]:
tested['orf'] = tested['ORF name'].astype(str)

In [23]:
tested['orf'] = clean_orf(tested['orf'])

In [24]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [25]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             record no. ORF name strain batch  plate  row   column  \
index_input                                                          
4667                NaN      NaN    NaN   NaN    NaN  NaN  end 150   

             OD BA/OD kan  orf  
index_input                     
4667                  NaN  NAN  


In [26]:
tested = tested.loc[t,:]

In [27]:
tested_orfs = tested['orf'].unique()

In [28]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [29]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [30]:
data = original_data.copy()

In [31]:
dataset_ids = [424]
datasets = datasets.reindex(index=dataset_ids)

In [32]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [33]:
data.head()

dataset_id,424
data_type,value
orf,
YAL068C,0
YAL067C,0
YAL066W,0
YAL065C,0
YAL062W,0


## Subset to the genes currently in SGD

In [34]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 27


In [35]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,424
,data_type,value
gene_id,orf,
1869,YAL068C,0
61,YAL067C,0
60,YAL066W,0
1727,YAL065C,0
57,YAL062W,0


# Normalize

In [36]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [37]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [38]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        424       
data_type       value valuez
gene_id orf                 
1869    YAL068C     0    0.0
61      YAL067C     0    0.0
60      YAL066W     0    0.0
1727    YAL065C     0    0.0
57      YAL062W     0    0.0

# Print out

In [39]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [40]:
from IO.save_data_to_db3 import *

In [41]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 22902726...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.29s/it]

Updating the data_modified_on field...
